# Analisando os dados da Olist

Para a nossa segunda parte da aula, iremos trabalhar em cima de uma base de dados disponibilizada pela Olist, uma empresa de e-commerce. Essa base de dados
é composta por 9 tabelas, conforme o seguinte esquema:

<center>
<img src=banco_de_dados_olist.png width=700>
</center>

Essa base de dados é uma base mais segmentada e é o que mais se aproxima da análise de dados do dia-a-dia de uma pessoa que trabalha com dados no mundo corporativo. Com essas tabelas, precisaremos fazer manipulações de dados de forma mais minuciona que aquela que fizemos pela manhã. Vamos lá!

Para ter acesso à base de dados, acesse: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce, ou baixe através do link:

 https://drive.google.com/drive/folders/1RuibSOdMp-cR6niAdbH8bRIj3PO4EOlh?usp=sharing


# Coleta dos Dados

Nossa base de dados da Olist é composta por 9 tabelas diferentes, nelas temos informações de:
- pedidos (olist_orders_dataset)
- itens dos pedidos (olist_order_items_dataset)
- review dos usuários sobre os pedidos (olist_order_reviews_dataset)
- detalhes de pagamento dos pedidos (olist_order_payments_dataset)
- detalhes do consumidor que fez os pedidos (olist_customers_dataset)
- detalhes de geolocalização do consumidor (olist_geolocation_dataset)
- detalhes dos produtos (olist_products_dataset)
- detalhes dos vendedores (olist_sellers_dataset)

Nesta aula, iremos focar em três datasets, o de pedidos (olist_orders_dataset), itens do pedido (olist_order_items_dataset) e o de consumidor (olist_customers_dataset). Vamos lá!

In [ ]:
import pandas as pd

Para a tabela de pedidos, iremos utilizar as seguintes colunas:

- **order_id**: Identificador do pedido
- **customer_id**: Identificador do cliente. Essa coluna é importante pois será a nossa chave com a tabela de consumidor
- **order_status**: Status do pedido
- **order_purchase_timestamp**: timestamp que a compra foi feita
- **order_estimated_delivered_date**: data prevista para a entrega da compra ao cliente
- **order_delivered_customer_date**: timestamp que a compra foi entregue ao cliente

In [ ]:
df_pedidos = pd.read_csv('dados/olist_orders_dataset.csv')
df_pedidos

Vamos ver como estão nossos dados com o `describe`?

In [ ]:
df_pedidos.describe(include='all')

Para a tabela de itens dos pedidos, iremos utilizar as seguintes colunas:

- **order_id**: Identificador do pedido. Será a nossa chave com a tabela de pedidos
- **product_id**: Identificador do produto
- **seller_id**: Identificador do vendedor
- **shipping_limit_date**: timestamp limite para a entrega do pedido
- **price**: valor do pedido
- **freight_value**: valor do frete

In [ ]:
df_itens_pedido = pd.read_csv('dados/olist_order_items_dataset.csv')
df_itens_pedido

In [ ]:
df_itens_pedido.describe(include='all')

Para a tabela de consumidor, iremos utilizar as seguintes colunas:

- **customer_id**: Identificador do cliente. Essa coluna é importante pois será a nossa chave com a tabela de pedidos. 
- **customer_state**: Estado
- **customer_city**: Cidade

In [ ]:
df_consumidor = pd.read_csv('dados/olist_customers_dataset.csv')
df_consumidor

In [ ]:
df_consumidor.describe(include='all')

## Joins entre tabelas

<img src="joins.png" text="https://www.youtube.com/watch?v=F2szczrXXqQ&list=PLNcg_FV9n7qZJqrKcUUCWCWPYCrlcVm9v&index=35" width=700px>

In [ ]:
df_a = pd.DataFrame({'id_a': [1,2,3,4,8]})
df_a

In [ ]:
df_b = pd.DataFrame({'id_b': [1,2,3,4,5,6]})
df_b

In [ ]:
df_uniao = pd.merge(df_a, df_b, left_on='id_a', right_on='id_b', how='inner')
df_uniao

In [ ]:
df_esquerda = pd.merge(df_a, df_b, left_on='id_a', right_on='id_b', how='left')
df_esquerda

In [ ]:
df_direita = pd.merge(df_a, df_b, left_on='id_a', right_on='id_b', how='right')
df_direita

In [ ]:
df_tudo = pd.merge(df_a, df_b, left_on='id_a', right_on='id_b', how='outer')
df_tudo

Vamos, agora, juntar a base de pedidos com a de itens. Quais tipos de join são os mais indicados para utilizarmos? Quais colunas devemos usar nesse merge? 

**Atividade:**
Juntar a base de pedidos com a de consumidores. Quais tipos de join são os mais indicados para utilizarmos? Quais colunas devemos usar nesse merge? 

# Objetivo da nossa Análise Exploratória:

Lembra que pela manhã comentamos a necessidade de estabelecer nossas perguntas para direcionar nossas análises? Para essa segunda parte da aula, buscaremos responder perguntas simples com base nessas 3 tabelas:

- Qual a frequência de compra dos clientes na plataforma da Olist no ano de 2017?
- Qual é o valor médio das compras desses clientes?  
- O número de vendas varia de acordo com a época do ano?
- O tempo limite de entrega das compras está sendo respeitado?

# Limpando os dados da Olist

## Pedidos

In [ ]:
# Vamos entender um pouquinho melhor dos nossos dados
df_pedidos.info()

Estamos lidando com um dataset relativamente grande, com quase 100.000 linhas. Temos dados nulos nas colunas de data,
o que faz sentido, pois nem todo o pedido realizado foi devidamente pago e finalizado. Por isso, algumas informações não estarão disponíveis. Por exemplo: um pedido que teve o pagamento negado, não terá dados de entrega. 

In [ ]:
df_pedidos[df_pedidos.order_approved_at.isna()]

Vamos começar nossa limpeza filtrando somente as colunas que desejamos utilizar


In [ ]:
columns_drop = ['order_approved_at', 'order_delivered_carrier_date', 'order_estimated_delivery_date']


Agora iremos filtrar somente aqueles pedidos que foram finalizados, mas antes é importante entender todos os status disponíveis que temos na base

Filtrando apenas os pedidos que foram entregues


Para fecharmos a limpeza dessa tabela de pedidos, precisamos criar uma coluna de mês e data de compra! Uma das perguntas que precisamos responder é se existe sazonalidade nas vendas, então, criar vamos criar uma coluna com a data (sem o horário) e outra truncada no início do mês para facilitar bastante nosso trabalho na etapa de análise:

## Itens do Pedido

In [ ]:
# Vamos entender um pouquinho melhor dos nossos dados
df_itens_pedido.info()

Estamos lidando com um dataset um pouco maior que o de pedidos, o que faz sentido, pois nossa informação aqui está a nível de item, e um pedido pode ter 1 ou mais itens. Não temos dados nulos nessa tabela, mas isso não significa que aqui constam somente pedidos entregues, tá? Precisaremos filtrá-los em breve. 

## Consumidor

In [ ]:
# Vamos entender um pouquinho melhor dos nossos dados
df_consumidor.info()

In [ ]:
df_consumidor.head()

Se nós repararmos na coluna `customer_zip_code_prefix`, o pandas a reconheceu como numérica, mas na verdade ela deveria ser uma string. Poderíamos utilizar o método `zfill` para preencher os valores com zero, mas como não sabemos se eram zeros à direita ou à esquerda a melhor solução é importar esses dados forçando que a coluna seja uma string. Para isso iremos utilizar o parâmetro `dtype` dentro do método `read_csv`.

In [ ]:
df_consumidor = pd.read_csv('dados/olist_customers_dataset.csv', dtype={"customer_zip_code_prefix": "string"})
df_consumidor

Essa coluna `customer_unique_id` não parece acrescentar nenhum informação, por isso vamos dropá-la!

In [ ]:
df_consumidor = df_consumidor.drop('customer_unique_id', axis=1)
df_consumidor

## Juntando os três datasets

Agora que fizemos a limpeza dos nossos três datasets, chegou a hora de juntarmos de acordo com a coluna em comum em cada um deles. Um ponto importante 
aqui é que queremos manter as informações que correspondem somente à pedidos **finalizados**. 

Para unir os 3 datasets usaremos o método `merge()`, precisamos entender qual é a coluna em comum entre os 3: no nosso caso, serão as colunas customer_id e order_id. 



In [ ]:
# Jutando os df pedidos e itens de pedidos


In [ ]:
# Jutando agora com os dados de consumidor


In [ ]:
# Exportando a base final
df_olist.to_csv('base_final_s14_olist.csv', index=False)

# Analisando os dados

Agora chegou a hora de respondermos as perguntas que estipulamos no começo dessa análise:
- Qual é o valor médio das compras desses clientes que tiveram os pedidos entregues?  
- O número de vendas finalizadas varia de acordo com a época do ano?
- O tempo limite de entrega das compras está sendo respeitado?

### Qual é o valor médio das compras desses clientes? E qual o valor médio pago em frete?


In [ ]:
# Aqui estamos buscando uma estatística descritiva, podemos utilizar a nossa função describe() para trazer essas informações


Vamos dar uma olhada como é a distribuição dos preços?

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

O `histplot` divide os dados em intervalos (bins) e conta quantas observações caem em cada intervalo, criando um gráfico de barras que mostra a distribuição da variável, o famoso histograma.

Ele é utilizado para analisar a **distribuição de variáveis numéricas**.

Apesar de termos pedidos com valores bem altos, como o caso do pedido no valor de R$ 6735, a média dos pedidos está cerca de 120 reais. O frete segue o mesmo padrão, sendo cerca de 10% do valor do pedido. 

Fazendo uma análise adicional aqui, pela coluna order_item_id, conseguimos observar a quantidade de itens que temos dentro de um pedido. A média de cada um desses pedidos é ter cerca de 1,2 itens. Sendo o seu máximo, um pedido que possui 21 itens iguais. 

### O número de vendas varia de acordo com a época do ano?

<img src=calendario-sazonalidades.jpg width=600>

Fonte: https://pagar.me/blog/como-lidar-com-a-sazonalidade-de-vendas-no-e-commerce/


Agora, vamos fazer uma análise de sazonalidade. Será que o número de vendas aumenta ou diminui de acordo com a época do ano?


O `lineplot` é uma função usada para criar gráficos de linha, sendo particularmente útil para visualizar séries temporais, que são dados coletados em intervalos de tempo sequenciais. Gráficos de linha são uma escolha comum para representar como uma variável numérica muda ao longo do tempo.

Considerações: Se os dados forem coletados em diferentes frequências (diária, semanal, mensal), certifique-se de que eles estejam agregados ou preparados corretamente antes de criar o gráfico e certifique-se que os dados aparecem em ordem cronológica.

Não parece existir um padrão de vendas ao longo dos anos e a tendência de crescimento da curva, pode dificultar a visualização de uma sazonalidade mensal. Vamos agora analisar as vendas por dia.

Olhando esse gráfico o que podemos observar?

Parece que temos uma sazonalidade semanal nas vendas. Quais são os dias com mais e menos vendas?

Vamos agora agrupar nossos dados pelos meses do ano e ver se tem algum mês com mais frequência de vendas:

As vendas parecem cair nos meses finais do ano. Isso acontece em todos os anos?

Como podemos ver, os dados não estão completos nem para 2016 e nem 2018, dificultando nossa análise sobre quais meses vendem mais. O fato de termos uma tendência de crescimento e não termos os meses finais de 2018, fez com que o gráfico anterior aparentasse que havia uma maior venda nos meses iniciais dos anos. E essa conclusão estava errada.

### O tempo limite de entrega das compras está sendo respeitado?
Para essa análise, iremos criar uma classificação para o status do frete, sendo: ATRASADO, ADIANTADO e DENTRO DO ESPERADO utilizando as colunas `shipping_limit_dt` e `order_delivered_date`, e então iremos observar a proporção dos pedidos para cada status.

In [ ]:
df_olist['order_delivered_customer_date'] = pd.to_datetime(df_olist['order_delivered_customer_date'])
df_olist['order_delivered_date'] = df_olist['order_delivered_customer_date'].dt.date

df_olist['shipping_limit_dt'] = pd.to_datetime(df_olist['shipping_limit_date']).dt.date

# condicional 
df_olist.loc[df_olist['order_delivered_date'] > df_olist['shipping_limit_dt'], 'status_entrega'] = 'ATRASADO'
df_olist.loc[df_olist['order_delivered_date'] < df_olist['shipping_limit_dt'], 'status_entrega'] = 'ADIANTADO'
df_olist.loc[df_olist['order_delivered_date'] == df_olist['shipping_limit_dt'], 'status_entrega'] = 'DENTRO DO ESPERADO'
df_olist

In [ ]:
df_olist["status_entrega"].value_counts(normalize=True)*100

Ou seja, cerca de 72% dos pedidos chegam atrasados na casa dos consumidores. É interessante entender o padrão aqui, esse status se mantém de acordo com a cidade? O quanto esse status mudaria se olhassemos para uma cidade grande tipo São Paulo? E por mês? Esses atrasos estão aumentando, diminuindo ou estão estáveis ao longo do tempo?

# Dinâmica: Explorando os dados da Olist

Para essa dinâmica, vocês terão 30 minutos para continuar explorando a base que criamos durante essa aula. A ideia é que vocês estipulem 3 a 5 perguntas simples (diferentes da que fizemos em aula) e faça as análises necessárias para chegar em sua resposta. 

# Dinâmica: Visualizações dos dados da Olist

Para essa dinâmica, vocês terão 40 minutos para criar as visualizações da base que criamos nessa aula (e com qualquer incremento que vocês fizeram à base na dinâmica anterior). 

Crie visualizações que te ajudem à responder as perguntas que vocês fizeram na dinâmica anterior. Utilize os conhecimentos da base da Netflix que fizemos pela manhã para criar seus próprios gráficos.


# Desafio da Semana

Objetivo: Sedimentar os conhecimentos de Análise de Dados que aprendemos nessa aula. 

Desafio: Criar um notebook de análise exploratória (como fizemos na nossa aula de hoje) com todas as etapas de coleta, limpeza, análise e visualização com base de dados da Olist.

Requisitos: 
- Formular o mínimo de 4 perguntas para responder com suas análises;
- Utilizar pelo menos 3 bases de dados da Olist (caso você deseje criar sua base do zero). Caso deseje continuar utilizando a que criamos em aula, é necessário incluir pelo menos mais 2 tabelas para enriquecer sua análise.   
- Criar pelo menos 3 gráficos.
- Exporte sua base final.


Submeta uma pasta que contenha: 
- o arquivo .ipynb com sua análise exploratória rodada, ou seja, com as respostas aparecendo no notebook;
- interpretações observadas a partir dos gráficos dentro do notebook;
- a base final criada por você no formato .csv;
- especificar quais bases da olist foram usadas;
